# Промо для расширения покупаемого ассортимента в рамках уже посещаемых отделов

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import warnings

In [2]:
df = pd.read_csv('test_data.csv')
df

,Клиент,Отдел,Количество,Сумма,Товар,Время_суток,Отдел_номер
0,456851.0,ОТД БАКАЛЕЯ/КОНСЕРВЫ,1.000,57.9900,"КОРОТКОРЕЗ(РОЖ,УЛИТ)",Утро/День,4
1,456851.0,ОТД БАКАЛЕЯ/КОНСЕРВЫ,1.000,57.9900,"ДЛИННОРЕЗ(СПАГ,МАК)",Утро/День,4
2,500944.0,ОТД КОНДИТЕРСКИЕ ИЗДЕ,0.387,325.0760,КОНФ ШОКОЛАДНЫЕ ВЕС,Утро/День,7
3,500944.0,ОТД КОНДИТЕРСКИЕ ИЗДЕ,0.302,233.1410,КОНФ ШОКОЛАДНЫЕ ВЕС,Утро/День,7
4,331038.0,ОТД МЯСНОЙ ЦЕХ,0.154,70.8385,BAР КОЛБ-НAРЕЗКA(SB),Утро/День,12
...,...,...,...,...,...,...,...
3961156,2190129.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3799.0000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2
3961157,347020.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3799.0000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2
3961158,567863.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3419.1000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2
3961159,305152.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3799.0000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2


In [3]:
data = df[['Клиент', 'Отдел', 'Товар','Количество','Сумма']].groupby(['Отдел', 'Клиент', 'Товар']).sum().reset_index()
data = data[data['Товар'] != 'ПАКЕТЫ']
data['Клиент'] = data['Клиент'].astype('int')
data

,Отдел,Клиент,Товар,Количество,Сумма
0,ОТД ABТОПРИНAДЛЕЖНОСТИ,2,КЛЕИ,2.0,41.98
1,ОТД ABТОПРИНAДЛЕЖНОСТИ,39,СЕМЕНA ГAЗОН ТРABЫ,6.0,12954.60
2,ОТД ABТОПРИНAДЛЕЖНОСТИ,49,КЛЕИ,2.0,56.98
3,ОТД ABТОПРИНAДЛЕЖНОСТИ,66,СЕМЕНA ОBОЩ КУЛЬТУР,4.0,30.56
4,ОТД ABТОПРИНAДЛЕЖНОСТИ,95,КЛЕИ,2.0,43.78
...,...,...,...,...,...
864850,ОТД ЭЛЕКТРОТОВАРЫ,3170939,LED-ЛAМПЫ,2.0,199.98
864851,ОТД ЭЛЕКТРОТОВАРЫ,3204725,LED-ЛAМПЫ,2.0,239.98
864852,ОТД ЭЛЕКТРОТОВАРЫ,3204725,БAТAРЕЙКИ AЛКAЛИН,2.0,139.98
864853,ОТД ЭЛЕКТРОТОВАРЫ,3204725,СТAНД ЛAМПЫ НAК Е14,2.0,79.98


# Рекомендации с учетом бюджета

Функция `get_department_frequent_items` используется для получения рекомендаций товаров по отделам для конкретного клиента. Основные шаги, которые она выполняет, следующие:

1. Определение отделов, в которых клиент делал покупки, путем извлечения уникальных значений отделов из данных, связанных с этим клиентом.
2. Фильтрация данных, чтобы исключить покупки клиента. То есть, удаляются записи о покупках клиента в отделах, которые он уже посещал.
3. Группировка данных по отделам и товарам с подсчетом частоты покупок каждого товара. Это позволяет определить наиболее популярные товары в каждом отделе.
4. Фильтрация товаров, которые клиент уже приобрел. Удаляются товары, которые клиент уже купил, чтобы предложить только новые товары.
5. Учет предпочтений клиента и его бюджета. Здесь учитывается средняя стоимость и среднее количество товаров в чеке, чтобы предложить товары, соответствующие предпочтениям клиента и его бюджету.
6. Фильтрация товаров по предпочтениям клиента и его бюджету. Удаляются товары, которые превышают бюджет клиента или не соответствуют его предпочтениям.
7. Выбор наиболее часто покупаемых товаров в каждом отделе в соответствии с количеством рекомендаций, указанным в `num_recommendations`.
8. Возврат полученных рекомендаций в виде результирующего набора данных.

Функция требует передачи идентификатора клиента, данных, количества рекомендаций и бюджета клиента. В приведенном примере кода также используются вспомогательные функции `calculate_customer_budget` для определения бюджета клиента и выводятся полученные рекомендации на экран.

In [4]:
# Функция для получения рекомендаций по отделам
def get_department_frequent_items(customer_id, data, num_recommendations, customer_budget):
    # Определение отделов, в которых клиент делал покупки
    customer_departments = data[data['Клиент'] == customer_id]['Отдел'].unique()

    # Фильтрация данных для исключения покупок клиента
    filtered_data = data[~((data['Клиент'] == customer_id) & (data['Отдел'].isin(customer_departments)))]
    
    # Группировка данных по отделам и подсчет частоты покупок каждого товара
    frequent_items = filtered_data.groupby(['Отдел', 'Товар']).size().reset_index(name='Частота').sort_values(by='Частота', ascending=False)
    
    # Фильтрация товаров, которые клиент уже приобрел
    customer_items = data[data['Клиент'] == customer_id]['Товар'].unique()
    frequent_items = frequent_items[~frequent_items['Товар'].isin(customer_items)]
    
    # Учет предпочтений клиента и его бюджета
    frequent_items = frequent_items.merge(data, on='Товар', how='left')
    frequent_items = frequent_items.groupby(['Отдел_x', 'Товар']).agg({
        'Частота': 'first',
        'Сумма': 'mean',  # Средняя стоимость товара
        'Количество': 'mean'  # Среднее количество товара в чеке
    }).reset_index()
    
    # Фильтрация товаров по предпочтениям клиента и его бюджету
    frequent_items = frequent_items[
        (frequent_items['Сумма'] <= customer_budget) &
        (frequent_items['Количество'] <= customer_budget)
    ]
    
    # Выбор наиболее часто покупаемых товаров в каждом отделе
    recommendations = frequent_items.groupby('Отдел_x').head(num_recommendations)
    
    return recommendations

Функция calculate_customer_budget вычисляет бюджет покупателя, то есть среднюю сумму, которую он тратит в магазине

In [5]:
def calculate_customer_budget(customer_id, data):
    customer_purchases = data[data['Клиент'] == customer_id]
    average_purchase_amount = customer_purchases['Сумма'].mean()
    return round(average_purchase_amount,2)

In [6]:
customer_id = int(input('ID клиента: '))
num_recommendations = 1

# Определение бюджета клиента
customer_budget = calculate_customer_budget(customer_id, data)

# Получение рекомендаций по отделам для выбранного клиента
recommendations = get_department_frequent_items(customer_id, data, num_recommendations, customer_budget)

# Вывод рекомендаций
print(f"Рекомендации для клиента {customer_id}:")
recommendations

ID клиента: 10
Рекомендации для клиента 10:


,Отдел_x,Товар,Частота,Сумма,Количество
9,ОТД ABТОПРИНAДЛЕЖНОСТИ,BСЕ Д/РОЗЖИГA,827,161.438235,4.640871
141,ОТД БАКАЛЕЯ/КОНСЕРВЫ,АССОРТИ ОВОЩНОЕ,4,126.750000,2.000000
270,ОТД ГАСТРОНОМИЯ,АЗИЯ-ВОК-ГРИЛЬ ПН,5,0.152000,0.800000
299,ОТД ИГРЫ/КАНЦТОВ,БУМ ПОДАРОЧ СТАНДАРТ,101,183.501683,2.306931
387,ОТД КОНДИТЕРСКИЕ ИЗДЕ,БАТОНЧИКИ-МЮСЛИ,929,208.408719,6.947255
462,ОТД КОСМЕТИКА/БЫТ.ХИМ�,ВАТНЫЕ ДИСКИ,190,167.442316,3.084211
607,ОТД МИР МАЛЫШЕЙ,AКСЕССУAРЫ ТЕКСТИЛЬ,30,161.646000,2.333333
646,ОТД МОЛОЧНЫЕ ПРОД.,ЙОГУРТ ВЯЗКИЙ ДЕТ,73,127.863288,6.000000
706,ОТД МЯСНОЙ ЦЕХ,BAР КОЛБAСA-НAРЕЗКA,546,167.366518,0.507641
723,ОТД НАПИТКИ,ВОДА ВКУСОВАЯ ПЭТ,51,124.677647,4.392157


В итоге эта функция предлагает клиенту во всех, отделах, в которых он когда-либо совершал покупки, продукты, которые являются наиболее продаваемыми и по стоимости соотносятся с теми, которые клиент уже когда-то приобретал.

# Рекомендации без учета бюджета

Данный метод является более удобным, потому что составляет топ отделов, в которых закупался клиент и дает рекомендации в рамках этих отделов. Удобно, что можно изначально задавать количество рекомендаций, как и в предыдущей функции, а также задавать количество отделов, которые мы берем из топа. Однако, функция не работает по принципу "два продукта из отдела 1, два продукта из отдела 2", а просто собирает набор из требуемого количества продуктов.

In [7]:
# Функция для получения рекомендаций на основе истории покупок клиента
# которая на вход требует номер клиента, данные, количество рекомендаций и количество отделовв из топа клиента
def get_recommendations(customer_id, data, num_recommendations,dep):
    # Шаг 1: Определение отделов, которые клиент посещает чаще всего
    customer_department_counts = data[data['Клиент'] == customer_id]['Отдел'].value_counts()
    frequent_departments = customer_department_counts.head(dep).index.tolist()

    # Шаг 2: Определение популярных товаров в выбранных отделах
    popular_items = data[data['Отдел'].isin(frequent_departments)]['Товар'].value_counts().index.tolist()

    # Шаг 3: Исключение товаров, которые клиент уже приобрел
    customer_items = data[data['Клиент'] == customer_id]['Товар'].unique()
    available_items = [item for item in popular_items if item not in customer_items]

    # Шаг 4: Предложение клиенту популярных товаров с указанием отдела
    recommendations = []
    print(f"Рекомендации для клиента c ID {customer_id}:")
    for item in available_items[:num_recommendations]:
        department = data[data['Товар'] == item]['Отдел'].iloc[0]
        recommendation = f"{item} из отдела {department}"
        recommendations.append(recommendation)
    for i in recommendations:
        print(i)

    return 


In [10]:
get_recommendations(10, data, 10, 4)

Рекомендации для клиента c ID 10:
КОРОТКОРЕЗ(РОЖ,УЛИТ) из отдела ОТД БАКАЛЕЯ/КОНСЕРВЫ
СВЕКЛА из отдела ОТД ОВОЩИ/ФРУКТЫ
СЫРКИ ГЛАЗИРОВАННЫЕ из отдела ОТД МОЛОЧНЫЕ ПРОД.
ГРЕЧКА ФАС из отдела ОТД БАКАЛЕЯ/КОНСЕРВЫ
МАЙОНЕЗ из отдела ОТД МОЛОЧНЫЕ ПРОД.
ДЛИННОРЕЗ(СПАГ,МАК) из отдела ОТД БАКАЛЕЯ/КОНСЕРВЫ
ГОРОШЕК из отдела ОТД БАКАЛЕЯ/КОНСЕРВЫ
МАСЛО С ДОБ ОЛИВКОВО из отдела ОТД БАКАЛЕЯ/КОНСЕРВЫ
АПЕЛЬСИНЫ из отдела ОТД ОВОЩИ/ФРУКТЫ
ПЕРСИКИ из отдела ОТД БАКАЛЕЯ/КОНСЕРВЫ
